In [2]:
# Calcula la temperatura y  
# precipitación media
# climatológica.

import os

import pandas as pd
import numpy as np

import xarray as xr

In [3]:
path_d = ["../../../resultados/CHIRTS/",
    "../../../datos/CHIRPS/"]

path = "../../../resultados/"

path_r = path + "vars/Actual/"

# Variables de CHIRTS/CHIRPS.
vars = ["Tmax", "Tmin", "Tmean",
    "Pre", "Pre_Tmean"]

# Si no existen las carpetas, las crea.
if not os.path.exists(path_r):
    os.mkdir(path_r)
for v in vars[0:4]:
    if not os.path.exists(path_r + v):
        os.mkdir(path_r + v)

In [10]:
# Cálculo de temperatura media y medias anuales.

# Creamos un Dataset sobre el que concatenaremos
# las temperaturas medias anuales.
with xr.open_dataset( path_d[0] + vars[0] + "/mexico_" 
    + vars[0] + ".1995.nc" ) as ds_max:
    # Creamos un Dataset con las dimensiones
    # adecuadas y un solo tiempo.
    Tmean = ( ds_max.isel(time = 0)
        .rename_vars(Tmax = vars[2]) )
    Tmean["time"] = np.datetime64("1994", "ns")
    Tmean = Tmean.where(Tmean > -9000)
    Tmean = ( Tmean
        .drop_vars("time")
        .assign_coords( {"dayofyear": 0}
        ) ).copy()
    Tmax = Tmean.rename_vars(
        Tmean = "Tmax" ).copy()
    Tmin = Tmean.rename_vars(
        Tmean = "Tmin" ).copy()
    T = [Tmax, Tmin, Tmean]
    Tmean = ( Tmean
        .drop_vars("dayofyear")
        .assign_coords( {"month": 0}
        ) ).copy()
    Tmax = Tmean.rename_vars(
        Tmean = "Tmax" ).copy()
    Tmin = Tmean.rename_vars(
        Tmean = "Tmin" ).copy()
    T_m = [Tmax, Tmin, Tmean]

# Calculamos la temperatura media anual.
for j in range(1995, 2017):
    # Si no existe la carpeta, la crea.
    if not os.path.exists(path_d[0] + vars[2]):
        os.mkdir(path_d[0] + vars[2])
    
    # Abrimos la temperatura máxima.
    with xr.open_dataset( path_d[0] +
        vars[0] + "/mexico_"  + vars[0] +
        "." + str(j) + ".nc" ) as ds_max:
        
        # Establecemos np.nan como valor sin datos.
        ds_max = ds_max.where(ds_max > -9000)
        
        # Obtenemos y guardamos
        # el promedio mensual.
        ds_month = ds_max.groupby(
            "time.month").mean("time")
        ds_month.to_netcdf( path_r + vars[0]
            + "/mexico_"  + vars[0] + "."
            + str(j) + "_monthly.nc" )
        ds_i_m = [ds_month.copy()]
        ds_year = ds_month.mean("month")
        ds_year.to_netcdf( path_r + vars[0]
            + "/mexico_"  + vars[0] + "."
            + str(j) + "_yearly.nc" )

        # Abrimos la temperatura mínima.
        with xr.open_dataset( path_d[0] +
            vars[1] + "/mexico_" + vars[1] +
            "." + str(j) + ".nc" ) as ds_min:
            
            # Establecemos np.nan como valor sin datos.
            ds_min = ds_min.where(ds_min > -9000)

            # Obtenemos y guardamos
            # el promedio mensual.
            ds_month = ds_min.groupby(
                "time.month").mean("time")
            ds_month.to_netcdf( path_r + vars[1]
                + "/mexico_"  + vars[1] + "."
                + str(j) + "_monthly.nc" )
            ds_i_m.append(ds_month.copy())
            ds_year = ds_month.mean("month")
            ds_year.to_netcdf( path_r + vars[1]
                + "/mexico_"  + vars[1] + "."
                + str(j) + "_yearly.nc" )

            # Calculamos la temperatura media.
            ds_mean = ( ( ds_max.rename_vars(
                Tmax = vars[2]) )
                + ( ds_min.rename_vars(
                Tmin = vars[2]) ) ) / 2

            # Guardamos el archivo diario.
            ds_mean.to_netcdf( path_d[0]
                + vars[2] + "/mexico_" +
                vars[2] + "." + str(j) + ".nc")

            # Obtenemos y guardamos
            # el promedio mensual.
            ds_month = ds_mean.groupby(
                "time.month").mean("time")
            ds_month.to_netcdf( path_r + vars[2]
                + "/mexico_"  + vars[2] + "."
                + str(j) + "_monthly.nc" )
            ds_i_m.append(ds_month.copy())
            ds_year = ds_month.mean("month")
            ds_year.to_netcdf( path_r + vars[2]
                + "/mexico_"  + vars[2] + "."
                + str(j) + "_yearly.nc" )
            
            ds_i = [ds_max, ds_min, ds_mean]

            if j not in [2015, 2016]:
                for i in range(3):
                    # Retiramos los días bisiestos.
                    ds_leap = ( ds_i[i].sel( 
                        time=~( (ds_i[i].time.dt.month == 2)
                        & (ds_i[i].time.dt.day == 29) ) ) )
                    # Cambiamos a un año que no sea bisiesto.
                    ds_leap["time"] = pd.date_range(
                        "1995-01-01", "1995-12-31",
                        freq = "1D" )
                    # Obtenemos el promedio por día del año.
                    ds_leap = ( ds_leap.groupby(
                        "time.dayofyear").mean("time") )
                    # Concatenamos las temperaturas.
                    T[i] = ( xr.concat( [T[i], ( ds_leap
                        ) ], dim = "dayofyear" ) )
                    # Si no es la primera iteración,
                    # obtenemos el promedio por día del año.
                    if (j != 1995):
                        T[i] = ( T[i]
                            .groupby("dayofyear")
                            .mean("dayofyear") )

                    # Concatenamos las temperaturas.
                    T_m[i] = ( xr.concat( [T_m[i], ( ds_i_m[i]
                        ) ], dim = "month" ) )
                    
                    # Si no es la primera iteración,
                    # obtenemos el promedio por día del año.
                    if (j != 1995):
                        T_m[i] = ( T_m[i]
                            .groupby("month")
                            .mean("month") ) 
            
for i in range(3):
    # Eliminamos el primer valor de tiempo,
    # que solo sirvió como base para concatenar
    # y obtenemos el promedio temporal.
    T[i] = T[i].drop_sel(dayofyear = 0)
    # Cambiamos el índice de día a fecha.
    T[i]["dayofyear"] = pd.date_range("1995-01-01", 
        "1995-12-31", freq = "1D")
    T[i] = T[i].rename_vars(dayofyear = "time")
    T[i] = T[i].swap_dims(dayofyear = "time")
    # Guardamos el archivo diario.
    T[i].to_netcdf(path_r + "mexico_"
        + vars[i] + "_daily.nc")

    # Eliminamos el primer valor de tiempo,
    # que solo sirvió como base para concatenar
    # y obtenemos el promedio temporal.
    T_m[i] = T_m[i].drop_sel(month = 0)
    # Cambiamos el índice de día a fecha.
    T_m[i]["month"] = pd.date_range("1995-01-01", 
        "1995-12-31", freq = "1M")
    T_m[i] = T_m[i].rename_vars(month = "time")
    T_m[i] = T_m[i].swap_dims(month = "time")
    # Guardamos el archivo diario.
    T_m[i].to_netcdf(path_r + "mexico_"
        + vars[i] + "_monthly.nc")
    # Guardamos el archivo promedio.
    T_m[i] = T_m[i].mean(dim = "time")
    T_m[i].to_netcdf(path_r + "mexico_"
        + vars[i] + "_yearly.nc")

In [66]:
# Cálculo de precipitación media anual.

# Creamos un Dataset sobre el que concatenaremos
# la precipitación media anual.
with xr.open_dataset( path_d[1] + "/mexico_" 
    + vars[3] + ".1995.nc" ) as ds_pre:
    # Creamos un Dataset con las dimensiones
    # adecuadas y un solo tiempo.
    Pre = ds_pre.isel(time = 0)
    Pre["time"] -= 1
    Pre = Pre.where(Pre > -9000)
    Pre = ( Pre
        .drop_vars("time")
        .assign_coords( {"dayofyear": 0}
        ) )
    Pre_m = ( Pre
        .drop_vars("dayofyear")
        .assign_coords( {"month": 0}
        ) ).copy()

# Calculamos la precipitación media anual.
for j in range(1995, 2017):
    
    # Abrimos la precipitación.
    with ( xr.open_dataset( path_d[1] + "/mexico_" 
        + vars[3] + "." + str(j) + ".nc" ) ) as ds_pre:
        
        # Establecemos np.nan como valor sin datos.
        ds_pre = ds_pre.where(ds_pre > -9000)

        # Obtenemos y guardamos
        # el promedio mensual.
        ds_month = ds_pre.groupby(
            "time.month").mean("time")
        ds_month.to_netcdf( path_r + vars[3]
            + "/mexico_"  + vars[3] + "."
            + str(j) + "_monthly.nc" )
        ds_year = ds_month.mean("month")
        ds_year.to_netcdf( path_r + vars[3]
            + "/mexico_"  + vars[3] + "."
            + str(j) + "_yearly.nc" )

        if j not in [2015, 2016]:
            # Retiramos los días bisiestos.
            ds_leap = ( ds_pre.sel( 
                time=~( (ds_pre.time.dt.month == 2)
                & (ds_pre.time.dt.day == 29) ) ) )
            # Cambiamos a un año que no sea bisiesto.
            ds_leap["time"] = pd.date_range(
                "1995-01-01", "1995-12-31", freq = "1D")
            # Obtenemos el promedio por día del año.
            ds_leap = ( ds_leap.groupby(
                "time.dayofyear").mean("time") )
            # Concatenamos las precipitaciones.
            Pre = ( xr.concat( [ Pre, ds_leap
                ], dim = "dayofyear" ) )
            # Si no es la primera iteración,
            # obtenemos el promedio por día del año.
            if (j != 1995):
                Pre = ( Pre
                    .groupby("dayofyear")
                    .mean("dayofyear") ) 

            # Concatenamos las temperaturas.
            Pre_m = ( xr.concat( [Pre_m, ( ds_month
                ) ], dim = "month" ) )
            # Si no es la primera iteración,
            # obtenemos el promedio por día del año.
            if (j != 1995):
                Pre_m = ( Pre_m
                    .groupby("month")
                    .mean("month") )       

# Eliminamos el primer valor de tiempo,
# que solo sirvió como base para concatenar.
Pre = Pre.drop_sel(dayofyear = 0)
# Cambiamos el índice de día a fecha.
Pre["dayofyear"] = pd.date_range("1995-01-01", 
        "1995-12-31", freq = "1D")
Pre = Pre.rename_vars(dayofyear = "time")
Pre = Pre.swap_dims(dayofyear = "time")
# Guardamos el archivo diario.
Pre.to_netcdf( path_r + "mexico_"
    + vars[3] + "_daily.nc" )

# Eliminamos el primer valor de tiempo,
# que solo sirvió como base para concatenar
# y obtenemos el promedio temporal.
Pre_m = Pre_m.drop_sel(month = 0)
# Cambiamos el índice de día a fecha.
Pre_m["month"] = pd.date_range("1995-01-01", 
    "1995-12-31", freq = "1M")
Pre_m = Pre_m.rename_vars(month = "time")
Pre_m = Pre_m.swap_dims(month = "time")
# Guardamos el archivo diario.
Pre_m.to_netcdf(path_r + "mexico_"
    + vars[3] + "_monthly.nc")
# Guardamos el archivo promedio.
Pre_m = Pre_m.mean(dim = "time")
Pre_m.to_netcdf( path_r +
    "mexico_" + vars[3]
    + "_yearly.nc" )